In [17]:
from langgraph.graph import StateGraph
#from langchain.llms import Bedrock
from langchain_community.chat_models import BedrockChat
from langchain_community.llms import Bedrock
from langchain_aws import BedrockLLM

import boto3
import json

import os
from dotenv import load_dotenv

load_dotenv()
os.environ["AWS_ACCESS_KEY_ID"]=os.getenv("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"]=os.getenv("AWS_SECRET_ACCESS_KEY")
os.environ["AWS_REGION"]=os.getenv("AWS_REGION")

# Initialize AWS Clients
sns_client = boto3.client("sns")
s3_client = boto3.client("s3")
logs_client = boto3.client("logs")
log_group_name = "arn:aws:logs:us-east-1:050451365316:log-group:/aws/lambda/test1:*"

# Amazon Titan Model
llm = BedrockLLM(
    model_id="amazon.titan-text-express-v1",
    client=boto3.client(service_name="bedrock-runtime")
)

c:\Users\dhire\miniconda3\envs\python3126\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_arn" in BedrockRerank has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [11]:
# Step 1: Read CloudWatch Logs
def read_cloudwatch_logs():
    response = logs_client.describe_log_streams(logGroupName="/aws/lambda/your_lambda_function")
    log_streams = response.get("logStreams", [])
    
    if log_streams:
        log_stream_name = log_streams[-1]["logStreamName"]
        log_events = logs_client.get_log_events(
            logGroupName="/aws/lambda/your_lambda_function",
            logStreamName=log_stream_name
        )
        logs = [event["message"] for event in log_events["events"]]
        return "\n".join(logs)
    return "No logs found."

In [12]:
# Step 2: Process Logs with LLM
def process_logs(logs):
    prompt = f"""Analyze the following CloudWatch logs and categorize errors as Critical, Warning, or Info. 
    Provide a summary of the errors found. Logs:
    {logs}"""
    response = llm.generate(prompt)
    return response

In [13]:
# Step 3: Store Logs in S3
def store_logs_in_s3(logs):
    bucket_name = "your-log-bucket"
    file_name = "processed_logs.json"
    s3_client.put_object(Bucket=bucket_name, Key=file_name, Body=json.dumps({"logs": logs}))
    return f"https://s3.amazonaws.com/{bucket_name}/{file_name}"

In [7]:
# Step 4: Send SNS Notification
def send_sns_notification(summary, s3_link):
    message = f"CloudWatch Error Summary:\n{summary}\n\nFull logs: {s3_link}"
    sns_client.publish(TopicArn="your-sns-topic-arn", Message=message, Subject="AWS Lambda Error Notification")

In [14]:
# Define Graph Workflow
graph = StateGraph(WorkflowState)
graph.add_node("read_logs", read_cloudwatch_logs)
graph.add_node("process_logs", process_logs)
graph.add_node("store_logs", store_logs_in_s3)
graph.add_node("send_notification", send_sns_notification)

graph.add_edge("read_logs", "process_logs")
graph.add_edge("process_logs", "store_logs")
graph.add_edge("store_logs", "send_notification")

graph.set_entry_point("read_logs")
workflow = graph.compile()


from IPython.display import Image, display
display(Image(workflow.get_graph().draw_mermaid_png()))


# Run the Workflow
workflow.invoke({"log_group_name": "/aws/lambda/test1", "logs": "", "processed_logs": "", "s3_link": ""})

NameError: name 'WorkflowState' is not defined